In [13]:
import numpy as np
import os
import glob
import itertools
import re
from scipy.fftpack import fft

In [14]:
def Data2ComplexBiasCorrected(filepath, dcI=2025, dcQ=2025):
    # Ref: https://github.com/dhruboroy29/MATLAB_Scripts/blob/6ded2d8d434239e3488ee79e02232dd7afff908c/Scripts/Data2IQ.m
    # Read IQ streams from data
    assert os.path.splitext(filepath)[1] == '.data' or os.path.splitext(filepath)[1] == '.bbs'
    comp = np.fromfile(filepath, dtype=np.uint16).astype(np.int16)
    I = comp[::2]
    Q = comp[1::2]
    assert len(I) == len(Q)

    # Sanity check of I and Q samples (>4096, or abruptly different from prev. sample?)
    for i in range(1,len(I)-1):
        if I[i]>4096 or abs(int(I[i])-int(I[i-1]))>2000 and abs(int(I[i])-int(I[i+1]))>1500:
            I[i] = I[i-1]
        if Q[i]>4096 or abs(int(Q[i])-int(Q[i-1]))>2000 and abs(int(Q[i])-int(Q[i+1]))>1500:
            Q[i] = Q[i-1]
    
    return (I-dcI) + 1j*(Q-dcQ)

In [15]:
def ComputeSTFT(X, stride=64, nfft=256, winlen=256):
    numsteps = (len(X)-winlen)//stride + 1
    
    # Prepare data for FFT
    Y=np.lib.stride_tricks.as_strided(X, (winlen,numsteps), strides=(X.itemsize, stride*X.itemsize))
    
    return fft(Y, n=nfft, axis=0)

## STFT Test Code

On MATLAB, variables saved as follows:
`save('trial.mat', '-v7.3')`

```
import h5py
f=h5py.File('trial.mat')

X = f['X'].value.view(np.complex)[0]
Z = ComputeSTFT(X)
```

Obtain MATLAB FFT to compare:

```
se = f['se'].value.view(np.complex)[0]
```

In [16]:
def extract_windows(file, winlen, stride=None):
        
    if stride is None:
        stride = winlen
    
    # Read complex data
    X = Data2ComplexBiasCorrected(file)
    
    numsteps = (len(X)-winlen)//stride + 1
    Y = np.lib.stride_tricks.as_strided(X, (numsteps,winlen), strides=(stride*X.itemsize, X.itemsize))
    
    # Validate windows
    j = 0
    for i in np.arange(len(Y)):
        assert (X[j*stride:j*stride+winlen] == Y[i]).all()
        j = j+1
        
    return X, Y

## Windows extraction test code

```
Data_cut, Data_win = extract_windows('/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/PedBike/final_bike_radial_full_cuts/a13_c1_Dec 9 2018_Bike_1p_cut4.data', 512)
```

In [17]:
bb_humans=list(itertools.chain(*[glob.glob(os.path.join(r, '*.data'))
                             for r in [
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/arc_1 (Humans_Gym balls)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/bv_4 (Humans_Cars)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/ceiling_238_10 (Humans_Gym balls)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/combined_5 (Humans_Dogs)/11-30-2011/Human',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/combined_5 (Humans_Dogs)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/kh_3 (Humans_Gym balls)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/prb_2 (Humans_Gym balls)/Human/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/Parking garage orthogonal (Humans)/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/Parking garage radial (Humans)/'
    ]]))
bb_nonhumans=list(itertools.chain(*[glob.glob(os.path.join(r, '*.data'))
                             for r in [
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/arc_1 (Humans_Gym balls)/Ball/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/bv_4 (Humans_Cars)/Car/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/ceiling_238_10 (Humans_Gym balls)/Ball/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/combined_5 (Humans_Dogs)/Dog/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/kh_3 (Humans_Gym balls)/Dog/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/prb_2 (Humans_Gym balls)/Dog/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/osu_farm_meadow_may24-28_2016_subset_113 (Cattle)/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/Radar_site1_hilltop (Cattle)/',
        '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Bumblebee/Targets/Radar_site2_creamery_subset_113 (Cattle)/'
    ]]))

aus_humans=list(itertools.chain(*[glob.glob(os.path.join(r, '*.data'))
                             for r in [
                                 '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Austere/BuildSys_Demo/Raw/final_human_full_cuts'
    ]]))

aus_nonhumans=list(itertools.chain(*[glob.glob(os.path.join(r, '*.data'))
                             for r in [
                                 '/mnt/6b93b438-a3d4-40d2-9f3d-d8cdbb850183/Research/Deep_Learning_Radar/Data/Austere/BuildSys_Demo/Raw/final_bike_radial_full_cuts'
    ]]))

# Window length
win_len = 512
win_stride = 128

all_datasets = [bb_humans, bb_nonhumans, aus_humans, aus_nonhumans]
processing_order = ['bb_humans', 'bb_nonhumans', 'aus_humans', 'aus_nonhumans']

In [20]:
for index in range(len(all_datasets)):
    files = all_datasets[index]
    print('Processing dataset:', processing_order[index])
    
    if index==0:
        out_folder = 'radar_classification_fft/Bumblebee/'
        flabel = 'Human'
    elif index==1:
        out_folder = 'radar_classification_fft/Bumblebee/'
        flabel = 'Nonhuman'
    elif index==2:
        out_folder = 'radar_classification_fft/Austere/'
        flabel = 'Human'
    elif index==3:
        out_folder = 'radar_classification_fft/Austere/'
        flabel = 'Nonhuman'
    
    os.makedirs(out_folder, exist_ok=True)
    
    Data_win_fft = []

    for cur_file in files:
        _, Data_win = extract_windows(cur_file, win_len, stride=win_stride)

        for index in np.arange(0, len(Data_win)):
            Data_win_fft.append(ComputeSTFT(Data_win[index]))
    
    # File order of processing, and extracted labels
    #print('{}\t{}'.format(fname, flabel))
    Data_win_fft = np.array(Data_win_fft)

    print(Data_win_fft.shape)
    
    np.save(os.path.join(out_folder, 'Data_win_fft_' + flabel + '.npy'), Data_win_fft)

Processing dataset: bb_humans
(8594, 256, 5)
Processing dataset: bb_nonhumans
(23594, 256, 5)
Processing dataset: aus_humans
(15021, 256, 5)
Processing dataset: aus_nonhumans
(7170, 256, 5)
